In [ ]:
import sys
sys.path.append("..")
import torch
import numpy as np
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from torch_utils.ambient_diffusion import nrmse_np, psnr

In [ ]:
inference_R = 4
training_R = 2
delta_prob = training_R+1
method = "A-ALD"

ssim_list = []
nrmse_list = []
psnr_list = []

for sample in tqdm(range(100)):
    if method == "A-DPS":
        results_file = '/csiNAS2/slow/brett/ambient_DPS_results/trained_r=%d_delta_prob%d/sample%d/seed%d/R=%d/checkpoint.pt'%(training_R, delta_prob, sample, 15, inference_R)
    if method == "A-ALD":
        results_file = '/home/asad/ambient-diffusion-mri/results/AmbientALD/trained_r=%d_delta_prob%d/sample%d/seed15/R=%d/checkpoint.pt'%(training_R, delta_prob, sample, inference_R)
    if method == "FS-DPS":
        results_file = '/csiNAS2/slow/brett/ambient_DPS_results/ambientmri_baseline_DPS_results_1_31_24/sample%d/R=%d/checkpoint.pt'%(sample,inference_R)
    elif method == "L1-DPS":
        results_file = '/csiNAS2/slow/brett/ambient_DPS_results/ambientmri_L1_R=%d_baseline_DPS_results_1_31_24/sample%d/R=%d/checkpoint.pt'%(training_R,sample,inference_R)
    
    cont = torch.load(results_file)
    gt_img = cont['gt_img']
    mean_recon = cont['recon']
    img_nrmse = nrmse_np(abs(gt_img[0,0]), abs(mean_recon[0,0]))
    img_SSIM = ssim(abs(gt_img[0,0]), abs(mean_recon[0,0]), data_range=abs(gt_img[0,0]).max() - abs(gt_img[0,0]).min())
    img_PSNR = psnr(gt=abs(gt_img[0,0]), est=abs(mean_recon[0]),max_pixel=np.amax(abs(gt_img)))

    nrmse_list.append(img_nrmse)
    ssim_list.append(img_SSIM)
    psnr_list.append(img_PSNR)

print('NRMSE: ', round(np.mean(nrmse_list), 3))
print('SSIM: ', round(np.mean(ssim_list)*100, 2))
print('PSNR: ', round(np.mean(psnr_list), 2))